In [185]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoSuchWindowException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [120]:
import time

In [205]:
url = 'https://www.instagram.com/eggsoeufs/'
options = webdriver.ChromeOptions()
options.add_argument(r"--user-data-dir=C:\Users\alexe\AppData\Local\Google\Chrome\User Data")
options.add_argument(r'--profile-directory=Default')
driver = webdriver.Chrome(executable_path=r"chromedriver.exe", options=options)
driver.get(url)

In [118]:
driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [206]:
last_height = driver.execute_script("return document.body.scrollHeight")
boolScroll = True
count = 4
list_links = []
while (boolScroll) and (count > 0):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    count = count - 1
    temp = driver.find_elements(By.XPATH,"//div[@class='_aabd _aa8k  _al3l']")
    for item in temp:
        try:
            temp1 = item.find_element(By.TAG_NAME, 'a')
            list_links.append(temp1.get_attribute('href'))
        except StaleElementReferenceException as Exception:
            print("error")
            temp1 = item.find_element(By.TAG_NAME, 'a')
            list_links.append(temp1.get_attribute('href'))
    time.sleep(5)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        boolScroll = False
        print(new_height)
    else:
        last_height = new_height
    print(last_height)

4165
5417
7608
8860


In [134]:
a = driver.find_elements(By.XPATH,"//div[@class='_ac7v  _al3n']")

In [207]:
print(len(list_links))

102


In [208]:
len(set(list_links))

66

In [202]:
list_links

['https://www.instagram.com/p/CrOUuNXArA2/',
 'https://www.instagram.com/p/Cq0kr_7A9Dp/',
 'https://www.instagram.com/p/CqBGhyKgJVs/',
 'https://www.instagram.com/p/CsJiqAZghe1/',
 'https://www.instagram.com/p/CsGWGbzgQLB/',
 'https://www.instagram.com/p/CsB0mAFgZeO/',
 'https://www.instagram.com/p/Cr8X5sOOpBN/',
 'https://www.instagram.com/p/Cr5VjgNs14w/',
 'https://www.instagram.com/p/Cr38xpyOjRI/',
 'https://www.instagram.com/p/Cr3TLqaMk3H/',
 'https://www.instagram.com/p/Cr08Ou5s2qT/',
 'https://www.instagram.com/p/CryXx9nMaPz/',
 'https://www.instagram.com/p/CsJiqAZghe1/',
 'https://www.instagram.com/p/CsGWGbzgQLB/',
 'https://www.instagram.com/p/CsB0mAFgZeO/',
 'https://www.instagram.com/p/Cr8X5sOOpBN/',
 'https://www.instagram.com/p/Cr5VjgNs14w/',
 'https://www.instagram.com/p/Cr38xpyOjRI/',
 'https://www.instagram.com/p/Cr3TLqaMk3H/',
 'https://www.instagram.com/p/Cr08Ou5s2qT/',
 'https://www.instagram.com/p/CryXx9nMaPz/',
 'https://www.instagram.com/p/CrvlPAutHrD/',
 'https://